# Dados

In [ ]:
# Dados para produtos discretos
idades <- 0:116
lx <- 100000 * (0.99)^idades
tábua_mortalidade <- data.frame(age = idades, lx = lx)

# Adicionar valores específicos
tábua_mortalidade$lx[tábua_mortalidade$age >= 109] <- c(1161, 552, 239, 92, 31, 9, 2, 0)

In [ ]:
# Parametos para produtos continuos
parametros <- data.frame(
  lambda = 0.5,
  gamma = 0.15,
  a = 0,
  b = 30,
  mu = 1000,
  sigma = 50
)

# Produtos para 1 vida

## Produtos discretos

In [ ]:
# Dotal puro
Ex <- function(n, i, x, df){

  # Função de desconto
  v <- 1 / (1 + i)

  # Cálculo do dote
  dote <- (v^n) * ((df[(x+n), 2])/(df[(x), 2]))

  return (dote)
}

In [ ]:
# Seguro
Ax1_discreto <- function(inicio_cobertura, final_cobertura, i, x, df) {

  # Função de desconto
  v <- 1 / (1 + i)

  # Cálculo do seguro
  vetor_soma <- numeric()
  for (t in inicio_cobertura:final_cobertura) {
      v_t <- v^(t + 1)
      t_q_x <- (df[(x + t), 2] - df[(x + t + 1), 2]) / df[x, 2]
      A_x <- v_t * t_q_x
      vetor_soma <- c(vetor_soma, A_x)}

  return(sum(vetor_soma, na.rm = TRUE))
}

In [ ]:
# Dotal misto
Ax_discreto <- function(inicio_cobertura, final_cobertura, i, x, df){

  # Função de desconto
  v <- 1 / (1 + i)

  # Calculo do seguro
  vetor_soma <- numeric()
  for (t in inicio_cobertura:final_cobertura) {
      v_t <- v^(t + 1)
      t_q_x <- (df[(x + t), 2] - df[(x + t + 1), 2]) / df[x, 2]
      A_x <- v_t * t_q_x
      vetor_soma <- c(vetor_soma, A_x)}

  # Calculo do dote puro
  n <- final_cobertura + 1
  dote <- (v^n) * ((df[(x+n), 2])/(df[(x), 2]))

  # Calculo do dotal misto
  dotal_misto <- vetor_soma + dote

  return(dotal_misto)

}

In [ ]:
# Anuidade
ax_discreto <- function(inicio_cobertura, final_cobertura, i, x, df) {

  # Função de desconto
  v <- 1 / (1 + i)

  # Cálculo da anuidade
  vetor_soma <- numeric()
  for (t in inicio_cobertura:final_cobertura) {
      v_t <- v^(t)
      t_p_x <- (df[(x + t), 2]) / (df[x, 2])
      a_x <- v_t * t_p_x
      vetor_soma <- c(vetor_soma, a_x)}

  return(sum(vetor_soma, na.rm = TRUE))
}

## Produtos continuos

In [ ]:
# Seguros
Ax_continuo <- function(inicio_cobertura, final_cobertura, i, x, dist, parametros) {

  # Função de desconto
  delta <- log(1 + i)

  # Formulando a integral com base na distribuição
  if (dist == "exp") {
    lambda <- parametros$lambda
    integrando <- function(t) {exp(-delta * t) * lambda * exp(-lambda * t)}
  }

  else if (dist == "unif") {
    a <- parametros$a
    b <- parametros$b
    integrando <- function(t) {exp(-delta * t) * (1 / (b - a))}
  }

  else if (dist == "weibull") {
    lambda <- parametros$lambda
    gamma <- parametros$gamma
    integrando <- function(t) {exp(-delta * t) * (lambda * gamma * (lambda * t)^(gamma - 1)) * exp(-(lambda * t)^gamma)}
  }

  else if (dist == "lognorm") {
    mu <- parametros$mu
    sigma <- parametros$sigma
    integrando <- function(t) {exp(-delta * t) * (1 / (t * sigma * sqrt(2 * pi)) * exp(-((log(t) - mu)^2) / (2 * sigma^2)))}
  }

  # Cálculo do seguro
  resultado <- integrate(
      f = integrando,
      lower = inicio_cobertura,
      upper = final_cobertura
  )

  return(resultado$value)
}

In [ ]:
# Anuidade
ax_continuo <- function(inicio_cobertura, final_cobertura, i, x, dist, parametros) {

  # Função de desconto
  delta <- log(1 + i)

  # Formulando a integral com base na distribuição
  if (dist == "exp") {
      lambda <- parametros$lambda
      integrando <- function(t) {exp(-delta * t) * exp(-lambda * t)}
  }

  else if (dist == "unif") {
      a <- parametros$a
      b <- parametros$b
      integrando <- function(t) {exp(-delta * t) * ((b - t) / (b - a))}
  }

  else if (dist == "weibull") {
      lambda <- parametros$lambda
      gamma <- parametros$gamma
      integrando <- function(t) {exp(-delta * t) * (exp(-(lambda * t)^gamma))}
  }

  else if (dist == "lognorm") {
    mu <- parametros$mu
    sigma <- parametros$sigma
    integrando <- function(t) {exp(-delta * t) * (1 - pnorm(log(t) / sigma))}
  }

  # Calculando a integral
  resultado <- integrate(
      f = integrando,
      lower = inicio_cobertura,
      upper = final_cobertura
  )

  return(resultado$value)
}

# Produtos para multiplas vidas

## Vida conjunta

In [ ]:
# Dotal puro
Exy_VC <- function(n, i, x, y, df){

  # Função de desconto
  v <- 1 / (1 + i)

  # Cálculo do dote
  n_p_x <- ((df[(x+n), 2])/(df[(x), 2]))
  n_p_y <- ((df[(y+n), 2])/(df[(y), 2]))
  dote <- (v^n) * n_p_x * n_p_y

  return (dote)
}

In [ ]:
# Seguro
Axy1_VC <- function(inicio_cobertura, final_cobertura, i, x, y, df) {

  # Função de desconto
  v <- 1 / (1 + i)

  # Cálculo do seguro
  vetor_soma <- numeric()
  for (t in inicio_cobertura:final_cobertura) {
      v_t <- v^(t + 1)
      t_q_x <- (df[(x + t), 2] - df[(x + t + 1), 2]) / df[x, 2]
      t_q_y <- (df[(y + t), 2] - df[(y + t + 1), 2]) / df[y, 2]
      Axy1 <- v_t * t_q_x * t_q_y
      vetor_soma <- c(vetor_soma, Axy1)}

  return(sum(vetor_soma, na.rm = TRUE))
}

In [ ]:
# Dotal misto
Axy_VC <- function(inicio_cobertura, final_cobertura, i, x, y, df){

  # Função de desconto
  v <- 1 / (1 + i)

  # Calculo do seguro
  vetor_soma <- numeric()
  for (t in inicio_cobertura:final_cobertura) {
      v_t <- v^(t + 1)
      t_q_x <- (df[(x + t), 2] - df[(x + t + 1), 2]) / df[x, 2]
      t_q_y <- (df[(y + t), 2] - df[(y + t + 1), 2]) / df[y, 2]
      Axy1 <- v_t * t_q_x * t_q_y
      vetor_soma <- c(vetor_soma, Axy1)}

  # Calculo do dote puro
  n <- final_cobertura + 1
  n_p_x <- ((df[(x+n), 2])/(df[(x), 2]))
  n_p_y <- ((df[(y+n), 2])/(df[(y), 2]))
  dote <- (v^n) * n_p_x * n_p_y

  # Calculo do dotal misto
  dotal_misto <- (sum(vetor_soma, na.rm = TRUE)) + dote

  return(dotal_misto)

}

In [ ]:
# Anuidade
axy_VC <- function(inicio_cobertura, final_cobertura, i, x, y, df) {

  # Função de desconto
  v <- 1 / (1 + i)

  # Cálculo da anuidade
  vetor_soma <- numeric()
  for (t in inicio_cobertura:final_cobertura) {
      v_t <- v^(t)
      t_p_x <- (df[(x + t), 2]) / (df[x, 2])
      t_p_y <- (df[(y + t), 2]) / (df[y, 2])
      a_xy <- v_t * t_p_x * t_p_y
      vetor_soma <- c(vetor_soma, a_xy)}

  return(sum(vetor_soma, na.rm = TRUE))
}

## Último sobrevivente

In [ ]:
# Dotal puro
Exy_US <- function(n, i, x, y, df){

  # Função de desconto
  v <- 1 / (1 + i)

  # Cálculo do dote
  n_p_x <- ((df[(x+n), 2])/(df[(x), 2]))
  n_p_y <- ((df[(y+n), 2])/(df[(y), 2]))

  dote <- (v^n) * ((n_p_x)+(n_p_y)-(n_p_x*n_p_y))

  return (dote)
}

In [ ]:
# Seguro
Axy1_US <- function(inicio_cobertura, final_cobertura, i, x, y, df) {

  # Função de desconto
  v <- 1 / (1 + i)

  # Cálculo do seguro
  vetor_soma <- numeric()
  for (t in inicio_cobertura:final_cobertura) {
      v_t <- v^(t + 1)
      t_q_x <- (df[(x + t), 2] - df[(x + t + 1), 2]) / df[x, 2]
      t_q_y <- (df[(y + t), 2] - df[(y + t + 1), 2]) / df[y, 2]
      Axy1 <- v_t * ((t_q_x)+(t_q_y)-(t_q_x * t_q_y))
      vetor_soma <- c(vetor_soma, Axy1)}

  return(sum(vetor_soma, na.rm = TRUE))
}

In [ ]:
# Dotal misto
Axy_US <- function(inicio_cobertura, final_cobertura, i, x, y, df){

  # Função de desconto
  v <- 1 / (1 + i)

  # Calculo do seguro
  vetor_soma <- numeric()
  for (t in inicio_cobertura:final_cobertura) {
      v_t <- v^(t + 1)
      t_q_x <- (df[(x + t), 2] - df[(x + t + 1), 2]) / df[x, 2]
      t_q_y <- (df[(y + t), 2] - df[(y + t + 1), 2]) / df[y, 2]
      Axy1 <- v_t * ((t_q_x)+(t_q_y)-(t_q_x * t_q_y))
      vetor_soma <- c(vetor_soma, Axy1)}

  # Calculo do dote puro
  n <- final_cobertura + 1
  n_p_x <- ((df[(x+n), 2])/(df[(x), 2]))
  n_p_y <- ((df[(y+n), 2])/(df[(y), 2]))

  dote <- (v^n) * ((n_p_x)+(n_p_y)-(n_p_x*n_p_y))

  # Calculo do dotal misto
  dotal_misto <- (sum(vetor_soma, na.rm = TRUE)) + dote

  return(dotal_misto)
}

In [ ]:
# Anuidade
axy_US <- function(inicio_cobertura, final_cobertura, i, x, y, df) {

  # Função de desconto
  v <- 1 / (1 + i)

  # Cálculo da anuidade
  vetor_soma <- numeric()
  for (t in inicio_cobertura:final_cobertura) {
      v_t <- v^(t)
      t_p_x <- (df[(x + t), 2]) / (df[x, 2])
      t_p_y <- (df[(y + t), 2]) / (df[y, 2])
      a_xy <- v_t * ((t_p_x)+(t_p_y)-(t_p_x * t_p_y))
      vetor_soma <- c(vetor_soma, a_xy)}

  return(sum(vetor_soma, na.rm = TRUE))

}

# Testes

In [1]:
# Parâmetros
n <- 3
i <- 0.04
x <- 111
y <- 110
inicio_cobertura <- 0
final_cobertura <- 2


# Chamar a função
resultado <- Axy_VC(inicio_cobertura = inicio_cobertura, final_cobertura = final_cobertura , i = i, x = x, y = y, df = tábua_mortalidade); resultado

ERROR: Error in Axy_VC(inicio_cobertura = inicio_cobertura, final_cobertura = final_cobertura, : could not find function "Axy_VC"
